---
#### update nav history downloader
---

In [1]:
import pandas as pd
import requests
import os

In [ ]:
#YYYY-MM-DD
def base_url(frm,to,amc_code): 
    return f"https://www.amfiindia.com/api/nav-history?query_type=historical_period&from_date={frm}&to_date={to}&sd_id={amc_code}"

In [ ]:
today = pd.Timestamp.today().date()
back_5yr  = (pd.Timestamp.today()-pd.Timedelta(days=365*5-7)).date()
back_10yr = (pd.Timestamp(str(back_5yr)) - pd.Timedelta(days=365*5-7)).date()
back_10yr

datetime.date(2015, 11, 3)

In [ ]:
table = pd.read_csv(r"core\SchemeData090825.csv")[["Code", "Scheme NAV Name","ISIN Div Payout/ ISIN GrowthISIN Div Reinvestment"]].astype(str)
table

,Code,Scheme NAV Name,ISIN Div Payout/ ISIN GrowthISIN Div Reinvestment
0,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165
1,100034,Aditya Birla Sun Life Large & Mid Cap Fund -Re...,INF209K01157INF209K01CE5
2,119433,Aditya Birla Sun Life Large & Mid Cap Fund -Di...,INF209K01VG0INF209K01VI6
3,119436,Aditya Birla Sun Life Large & Mid Cap Fund - G...,INF209K01VH8
4,100037,Aditya Birla Sun Life Income Fund - Regular - ...,INF209K01587INF209K01CY3
...,...,...,...
16064,153816,Mirae Asset Multi Factor Passive FOF - Direct ...,INF769K01PH3
16065,153817,Mirae Asset Multi Factor Passive FOF - Direct ...,INF769K01PJ9INF769K01PI1
16066,153818,Mirae Asset Multi Factor Passive FOF - Regular...,INF769K01PE0
16067,153819,Mirae Asset Multi Factor Passive FOF - Regular...,INF769K01PG5INF769K01PF7


In [ ]:
import re
def get_isin(string):
    if not isinstance(string, str):
        return [None, None]
    isin_pattern = "[A-Z]{3}[0-9A-Z]{9}"
    matches =  re.findall(isin_pattern, string)
    if(len(matches)) == 0: return [None, None]
    if(len(matches) == 1): matches.append(None)
    return matches

for idx , row in table.iterrows():
    matches = get_isin(row["ISIN Div Payout/ ISIN GrowthISIN Div Reinvestment"])
    table.at[idx,"ISIN Div Payout/ ISIN Growth"] = matches[0]
    table.at[idx, "ISIN Div Reinvestment"] = matches[1]

In [ ]:
table

,Code,Scheme NAV Name,ISIN Div Payout/ ISIN GrowthISIN Div Reinvestment,ISIN Div Payout/ ISIN Growth,ISIN Div Reinvestment
0,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,INF209K01165,None
1,100034,Aditya Birla Sun Life Large & Mid Cap Fund -Re...,INF209K01157INF209K01CE5,INF209K01157,INF209K01CE5
2,119433,Aditya Birla Sun Life Large & Mid Cap Fund -Di...,INF209K01VG0INF209K01VI6,INF209K01VG0,INF209K01VI6
3,119436,Aditya Birla Sun Life Large & Mid Cap Fund - G...,INF209K01VH8,INF209K01VH8,None
4,100037,Aditya Birla Sun Life Income Fund - Regular - ...,INF209K01587INF209K01CY3,INF209K01587,INF209K01CY3
...,...,...,...,...,...
16064,153816,Mirae Asset Multi Factor Passive FOF - Direct ...,INF769K01PH3,INF769K01PH3,None
16065,153817,Mirae Asset Multi Factor Passive FOF - Direct ...,INF769K01PJ9INF769K01PI1,INF769K01PJ9,INF769K01PI1
16066,153818,Mirae Asset Multi Factor Passive FOF - Regular...,INF769K01PE0,INF769K01PE0,None
16067,153819,Mirae Asset Multi Factor Passive FOF - Regular...,INF769K01PG5INF769K01PF7,INF769K01PG5,INF769K01PF7


In [ ]:
table.iloc[:,3:]

,ISIN Div Payout/ ISIN Growth,ISIN Div Reinvestment
0,INF209K01165,None
1,INF209K01157,INF209K01CE5
2,INF209K01VG0,INF209K01VI6
3,INF209K01VH8,None
4,INF209K01587,INF209K01CY3
...,...,...
16064,INF769K01PH3,None
16065,INF769K01PJ9,INF769K01PI1
16066,INF769K01PE0,None
16067,INF769K01PG5,INF769K01PF7


In [ ]:
def find_on_isin(row):
    payoutisin = row["ISIN Div Payout/ ISIN Growth"]
    reinvestisin = row["ISIN Div Reinvestment"]
    isin_to_find = ["INF846K01CB0", "INF179K01830", "INF209K011W7"]
    ans = payoutisin in isin_to_find or reinvestisin in isin_to_find
    return ans
subset_table=table[table.apply(find_on_isin, axis = 1)]

In [ ]:
base_url(frm = back_10yr, to=back_5yr, amc_code = "100039")

'https://www.amfiindia.com/api/nav-history?query_type=historical_period&from_date=2015-11-01&to_date=2020-10-23&sd_id=100039'

In [ ]:
resp = requests.get(base_url(frm = back_10yr, to=back_5yr, amc_code = "100039"))
resp.ok

True

In [ ]:
data = resp.json()
data

{'message': 'No records found for date range: 2015-11-01 to 2020-10-23'}

In [ ]:
def get_5yr_nav(code):
    url = base_url(back_5yr,today,code)
    resp = requests.get(url)
    if not resp.ok:
        return []
    data = resp.json()
    try:
        datapoints = data["data"]["nav_groups"][0]["historical_records"]
    except Exception as e:
        return[]
    return datapoints
def get_10yr_nav(code):
    url = base_url(back_10yr,back_5yr,code)
    resp = requests.get(url)
    if not resp.ok:
        return []
    data = resp.json()
    try:
        datapoints = data["data"]["nav_groups"][0]["historical_records"]
    except Exception as e:
        return[]
    return datapoints

In [ ]:
output_folder = "historical_nav_sepcified"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
tbl = subset_table
for idx,row in tbl.iterrows():
    code = row.get("Code","")
    name = row.get("Scheme NAV Name","")
    paygrow = row.get("ISIN Div Payout/ ISIN Growth","")
    reinvest = row.get("ISIN Div Reinvestment","")
    sale_price=""
    header = "Scheme Code;Scheme Name;ISIN Div Payout/ISIN Growth;ISIN Div Reinvestment;Net Asset Value;Repurchase Price;Sale Price;Date"
    text = header+"\n"
    
    dps_5yr = get_5yr_nav(code)
    dps_10yr = get_10yr_nav(code)
    data_points = dps_5yr + dps_10yr

    for record in data_points:
        date = record.get("date","")
        date = pd.Timestamp(date).strftime("%d-%b-%Y")
        nav = record.get("nav","")
        repurchase = record.get("repurchase","")
        new_record = [code, name, paygrow, reinvest, nav, repurchase, sale_price, date]
        filter  = lambda x: "" if not x else str(x)
        new_record = list(map(filter, new_record))
        new_line = ";".join(new_record)
        text+= "\n" + new_line
    filename = f"{name}.txt"
    filepath = os.path.join(output_folder, filename)
    with open(filepath, "w") as f:
        f.write(text)     

'26-Oct-2020'

---
---

---  
#### correct date time format  
---  

In [4]:
output_folder = "historical_nav"
filenames=os.listdir(output_folder)
filepath= list(map(lambda x: os.path.join(output_folder,x),filenames))

In [9]:
len(filepath)

6775

In [10]:
sum(list(map(lambda x : not x.lower().startswith("historical_nav\\franklin"),filepath)))

6541

In [12]:
for file in filepath[6500:]:
    try:
        df = pd.read_csv(file, delimiter=";")
        df["Date"] = df["Date"].apply(lambda x: pd.Timestamp(x).strftime("%d-%b-%Y"))
        text = ";".join(list(df.columns))
        for _,row in df.iterrows():
            vals = row.to_list()
            vals = list(map(lambda x: str(x) if not pd.isna(x) else "", vals))
            text +="\n"+";".join(vals)
        with open(file,"w") as f:
            f.write(text)
    except Exception as e:
        print(f"removing {file} as {str(e)}")
        inp = input().lower()
        if(inp!="n"):
            os.remove(file)
        


In [14]:
import os
files = os.listdir("historical_nav")

In [16]:
files.index("Franklin India Corporate Debt Fund - Plan B - Monthly - IDCW.txt")

1104

---
--- 

In [ ]:
file

,100033,Aditya Birla Sun Life Large & Mid Cap Fund - Regular Growth,INF209K01165,Unnamed: 3,423.2400,Unnamed: 5,Unnamed: 6,2020-10-23
0,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,NaN,417.47,NaN,NaN,2020-10-26
1,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,NaN,422.82,NaN,NaN,2020-10-27
2,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,NaN,417.80,NaN,NaN,2020-10-28
3,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,NaN,417.79,NaN,NaN,2020-10-29
4,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,NaN,416.80,NaN,NaN,2020-10-30
...,...,...,...,...,...,...,...,...
2447,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,NaN,418.66,NaN,NaN,2020-10-19
2448,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,NaN,420.80,NaN,NaN,2020-10-20
2449,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,NaN,421.53,NaN,NaN,2020-10-21
2450,100033,Aditya Birla Sun Life Large & Mid Cap Fund - R...,INF209K01165,NaN,420.46,NaN,NaN,2020-10-22


In [ ]:
import pandas as pd
df = pd.read_csv("nav_time_series.csv", delimiter=";")
isins_df = df["Scheme Code"].unique().tolist()

C:\Users\vaibh\AppData\Local\Temp\ipykernel_11328\266203744.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("nav_time_series.csv", delimiter=";")


In [ ]:
with open("tt.txt", "r") as f:
    isins_file = list(map(int,f.readline().split(",")))

In [ ]:
len(isins_df) , len(isins_file)

(21724, 21724)

In [ ]:
len(set(isins_file).difference(set(isins_df)))

0

In [ ]:
a = set(sorted(isins_df))

In [ ]:
b = set(sorted(isins_file))

In [ ]:
a_b = a.difference(b)
len(a_b)

16955

In [ ]:
b_a = b.difference(a)
len(b_a)

0

In [ ]:
import requests

# Target URL
url = "https://www.amfiindia.com/modules/NavHistoryAll"

# Headers to mimic browser
headers = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.amfiindia.com",
    "Referer": "https://www.amfiindia.com/net-asset-value/nav-history",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
}

# Payload with your desired date
data = {
    "fDate": "14-Jul-2025"  # <-- You can change this to any date you want
}

# Send the POST request
response = requests.post(url, headers=headers, data=data)

# Display the response content (HTML)
if response.status_code == 200:
    print("✅ Request successful!")
    print(response.text[:1000])  # Only print first 1000 chars for readability
else:
    print(f"❌ Request failed with status code {response.status_code}")


✅ Request successful!

                <table border="0" cellpadding="0" cellspacing="0" id="divDownloadsAll">
                    <tr>
                        <td>
                            <form action="/modules/_xlsDownload" method="post">	<input id="hdnVal" name="hdnVal" type="hidden" value="0" />	<input type="submit" class="serch-rgt" title="Excel"></form>
                            <form action="/modules/_wordDownload" method="post">	<input id="hdnWordVal" name="hdnWordVal" type="hidden" value="0" />	<input type="submit" class="word-serch-rgt" title="Word"></form>
                        </td>
                    </tr>
                </table>
                <div id="divExcelAll">
                    <table style="margin-top: -25px" class="nav-resut-tble" border="0" cellpadding="0" cellspacing="0">
                        <tbody>
                            <tr style="background-color: rgb(240, 240, 240);">
                                <th colspan="4">View all NAVs for all

In [ ]:
with open("tt.html","w") as f :
    f.write(response.text)

In [ ]:
respon